In [ ]:
%pip install google-generativeai langgraph tavily
%pip install duckduckgo-search
%pip install tavily-python
%pip install beautifulsoup4
%pip install ddgs


In [34]:
from tavily import TavilyClient
import os
import google.generativeai as genai
import requests
from bs4 import BeautifulSoup
from ddgs import DDGS
import re
from dotenv import load_dotenv
load_dotenv()

os.environ['GOOGLE_API_KEY'] = os.getenv('GEMINI_API_KEY')
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')

In [31]:
client = TavilyClient(api_key=os.environ.get("TAVILY_API_KEY"))

result = client.search("O que sao multiagentes de Inteligencia Artificial?", include_answer=True)
result["answer"]

'Sistemas multiagentes de Inteligência Artificial são conjuntos de agentes autônomos que trabalham juntos para resolver problemas complexos. Cada agente tem autonomia e coordenação para atingir objetivos comuns. Eles são usados para melhorar eficiência e inovação em várias aplicações.'

In [32]:
cidade = "Belém"

query = f"""
Liste os 5 principais restaurantes em {cidade}, segundo avaliações recentes no TripAdvisor ou sites similares de turismo.
Para cada restaurante, informe:
- Tipo de culinária (ex: regional, italiana, japonesa)
- Uma breve descrição (máx. 2 linhas)
- Avaliação média (se disponível)
- Faixa de preço

Responda apenas com dados atualizados e relevantes para turistas.
"""

In [36]:
ddg = DDGS()
def search(query, max_results=6):
    try:
        results = ddg.text(query, max_results=max_results)
        return [i["href"] for i in results]
    except Exception as e:
        raise e
        
for link in search(query):
    print(link)

https://indicados.com.br/melhores-restaurantes-de-belem/
https://exame.com/casual/os-melhores-restaurantes-de-belem-segundo-ranking-exame-casual-2025/
https://www.tripadvisor.com.br/Restaurants-g303404-Belem_State_of_Para.html
https://hoteisrestaurantes.com.br/artigos/melhores-restaurantes-em-belem/
https://www.temqueir.com.br/2021/05/onde-comer-em-belem-restaurantes.html
https://www.lisbonlux.com/lisboa/restaurantes/belem.html


In [ ]:
api_key = os.environ.get("TAVILY_API_KEY")
if not api_key:
    raise ValueError("A chave TAVILY_API_KEY não foi encontrada.")

cliente_tavily = TavilyClient(api_key=api_key)

cidade = "Belém do Pará"
tavily_query = f"restaurantes em {cidade} tripadvisor com maior quantidade de reviews e faixa de preço"

print("Iniciando busca agêntica por URLs do Tripadvisor com Tavily...")
tripadvisor_url = None
try:
    tavily_results = cliente_tavily.search(query=tavily_query, max_results=5)
    
    if tavily_results and tavily_results["results"]:
        print(f"Tavily encontrou {len(tavily_results['results'])} resultados. Analisando...")
        
        for result in tavily_results["results"]:
            url = result["url"]
            
            if "tripadvisor.com" in url or "tripadvisor.com.br" in url:
                tripadvisor_url = url
                break
        
        if not tripadvisor_url:
            print("Nenhum URL relevante do Tripadvisor foi encontrado nos primeiros resultados.")
    else:
        print("Tavily não encontrou resultados para a busca agêntica.")
            
except Exception as e:
    print(f"Erro na busca agêntica com Tavily: {e}. Verifique sua chave API ou conexão.")
    
if tripadvisor_url:
    clean_url = re.sub(r'-o\d+-', '-', tripadvisor_url)
    tripadvisor_url = clean_url
    print(f"✅ URL encontrada limpa de paginação.")
    
print("-" * 50)
print(f"URL Final do Tripadvisor para raspagem: {tripadvisor_url if tripadvisor_url else 'NÃO ENCONTRADO'}")
print("-" * 50)

In [ ]:
%pip install selenium
%pip install webdriver-manager

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.common.exceptions import WebDriverException, TimeoutException

def scrape_restaurantes_info(url):
    if not url:
        print("Erro: URL vazia ou não localizada para raspagem.")
        return None
    try:
        service = Service(ChromeDriverManager().install())
        options = webdriver.ChromeOptions()
        options.add_argument("--headless")
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126")
        
        driver = webdriver.Chrome(service=service, options=options)
        driver.set_page_load_timeout(30)
    except Exception as e:
        print(f'Erro ao inicializar o driver do Selenium: {e}')
        return None
    try:
        print(f"Tentando carregar a página com Selenium: {url}")
        driver.get(url)
        
        driver.implicitly_wait(10)
        
        response_text = driver.page_source
    except TimeoutException as e:
        print(f'Erro de tempo limite ao carregar a pagina {url}.')
        return None
    except WebDriverException as e:
        print(f" Erro ao carregar a pagina {url} com Selenium: {e}. Pode ser um bloqueio ou problema de conexão")
        return None
    finally:
        driver.quit()

    soup = BeautifulSoup(response_text, "html.parser")
    return soup

soup_tripadvisor = None    

soup_tripadvisor = None

if 'tripadvisor_url' in locals() and tripadvisor_url:
    print(f"\nTentando raspar a página identificada: {tripadvisor_url}")
    soup_tripadvisor = scrape_restaurantes_info(tripadvisor_url)
    if soup_tripadvisor:
        print("HTML da página do Tripadvisor obtido com sucesso!")
        page_title_tag = soup_tripadvisor.find('title')
        if page_title_tag:
            print(f"Título da página: {page_title_tag.get_text(strip=True)}")
        else:
            print("Não foi possível encontrar o título da página.")
    else:
        print("falha ao raspar a página do Tripadvisor. Verifique o URL ou se o site está bloqueando.")
else:
    print("Não há URL do tripadvisor válido para raspar (obtido no bloco 1).")
print("-"*50)


Tentando raspar a página identificada: https://www.tripadvisor.com.br/Restaurants-g303404-c22-Belem_State_of_Para.html
Tentando carregar a página com Selenium: https://www.tripadvisor.com.br/Restaurants-g303404-c22-Belem_State_of_Para.html
HTML da página do Tripadvisor obtido com sucesso!
Título da página: tripadvisor.com.br


In [41]:
from bs4 import BeautifulSoup
import re 

html_completo_exemplos = """
<div class="rfqBq f e"><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d24040497-Reviews-Camarada_Camarao_Belem_Shopping_Boulevard_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/25/2b/4e/d2/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/25/2b/4e/d2/caption.jpg?w=700&amp;h=700&amp;s=1 2x"><img width="400" height="400" alt="." loading="lazy" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/25/2b/4e/d2/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/25/2b/4e/d2/caption.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/25/2b/4e/d2/caption.jpg?w=400&amp;h=400&amp;s=1"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img width="400" height="260" alt="Burger Terra e Mar" loading="lazy" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/25/8e/92/a0/burger-terra-e-mar.jpg?w=400&amp;h=400&amp;s=1" style="" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/25/8e/92/a0/burger-terra-e-mar.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/25/8e/92/a0/burger-terra-e-mar.jpg?w=500&amp;h=500&amp;s=1 2x"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/25/2b/4f/07/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/25/2b/4f/07/caption.jpg?w=700&amp;h=700&amp;s=1 2x"><img width="400" height="400" alt="." loading="lazy" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/25/2b/4f/07/caption.jpg?w=400&amp;h=400&amp;s=1" style="" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/25/2b/4f/07/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/25/2b/4f/07/caption.jpg?w=800&amp;h=800&amp;s=1 2x"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-24040497"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d24040497-Reviews-Camarada_Camarao_Belem_Shopping_Boulevard_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">1. Camarada Camarão Belém - Shopping Boulevard Belém</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u Q1"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4,9</span></div></span><div class="nKWJn u Q1 qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby="_lithium-r_au_" data-automation="bubbleRatingImage"><title id="_lithium-r_au_">4,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d24040497-Reviews-Camarada_Camarao_Belem_Shopping_Boulevard_Belem-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(1.699 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Brasileira, Frutos do mar</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://livemenu.app/menu/627bcbac218bd50011827877" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Cardápio</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d24040497-Reviews-Camarada_Camarao_Belem_Shopping_Boulevard_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Maravilhosa</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d24040497-Reviews-Camarada_Camarao_Belem_Shopping_Boulevard_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Maravilhosa</span></div></a></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d14107327-Reviews-Restaurante_JP_Grill-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/ff/59/63/photo0jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/ff/59/63/photo0jpg.jpg?w=700&amp;h=-1&amp;s=1 2x"><img width="400" height="400" alt="" loading="lazy" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/ff/59/63/photo0jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/ff/59/63/photo0jpg.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/ff/59/63/photo0jpg.jpg?w=400&amp;h=-1&amp;s=1"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/05/30/a3/photo1jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/05/30/a3/photo1jpg.jpg?w=700&amp;h=-1&amp;s=1 2x"><img width="400" height="400" alt="" loading="lazy" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/05/30/a3/photo1jpg.jpg?w=400&amp;h=-1&amp;s=1" style="" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/05/30/a3/photo1jpg.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/15/05/30/a3/photo1jpg.jpg?w=800&amp;h=-1&amp;s=1 2x"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/31/e4/d9/9f/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/31/e4/d9/9f/caption.jpg?w=700&amp;h=700&amp;s=1 2x"><img width="400" height="400" alt="" loading="lazy" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/31/e4/d9/9f/caption.jpg?w=400&amp;h=400&amp;s=1" style="" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/31/e4/d9/9f/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/31/e4/d9/9f/caption.jpg?w=800&amp;h=800&amp;s=1 2x"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-14107327"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d14107327-Reviews-Restaurante_JP_Grill-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">2. Restaurante JP Grill</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u Q1"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4,8</span></div></span><div class="nKWJn u Q1 qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby="_lithium-r_b7_" data-automation="bubbleRatingImage"><title id="_lithium-r_b7_">4,8 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d14107327-Reviews-Restaurante_JP_Grill-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(12 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Brasileira, Internacional</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d14107327-Reviews-Restaurante_JP_Grill-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Vale o que custo</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d14107327-Reviews-Restaurante_JP_Grill-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Restaurante JP</span></div></a></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d21311161-Reviews-Mirakuru_Street_Food-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/1f/3f/28/e-bem-desse-jeito-mesmo.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/1f/3f/28/e-bem-desse-jeito-mesmo.jpg?w=700&amp;h=700&amp;s=1 2x"><img width="400" height="400" alt="É bem desse jeito mesmo: calçada esburacada e parede faltando reboco, mas garantimos que a comida é de qualidade." loading="lazy" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/1f/3f/28/e-bem-desse-jeito-mesmo.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/1f/3f/28/e-bem-desse-jeito-mesmo.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/1f/3f/28/e-bem-desse-jeito-mesmo.jpg?w=400&amp;h=400&amp;s=1"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/24/c3/3a/tantanmen-macarrao-caseiro.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/24/c3/3a/tantanmen-macarrao-caseiro.jpg?w=700&amp;h=-1&amp;s=1 2x"><img width="400" height="400" alt="Tantanmen - Macarrão caseiro, caldo de porco, ovo curado no shoyu, carne de pouco moída, cebolinha e bok choi" loading="lazy" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/24/c3/3a/tantanmen-macarrao-caseiro.jpg?w=400&amp;h=-1&amp;s=1" style="" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/24/c3/3a/tantanmen-macarrao-caseiro.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/24/c3/3a/tantanmen-macarrao-caseiro.jpg?w=800&amp;h=-1&amp;s=1 2x"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/24/c3/4b/chashu-shio-ramen-macarrao.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/24/c3/4b/chashu-shio-ramen-macarrao.jpg?w=700&amp;h=-1&amp;s=1 2x"><img width="400" height="400" alt="Chashu Shio Ramen - Macarrão caseiro, caldo de frango com porco, porco assado, milho queimado, cebolinha, ovo curado no shoyu" loading="lazy" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/24/c3/4b/chashu-shio-ramen-macarrao.jpg?w=400&amp;h=-1&amp;s=1" style="" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/24/c3/4b/chashu-shio-ramen-macarrao.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1c/24/c3/4b/chashu-shio-ramen-macarrao.jpg?w=800&amp;h=-1&amp;s=1 2x"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-21311161"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d21311161-Reviews-Mirakuru_Street_Food-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">3. Mirakuru Street Food</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u Q1"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>5,0</span></div></span><div class="nKWJn u Q1 qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby="_lithium-r_bg_" data-automation="bubbleRatingImage"><title id="_lithium-r_bg_">5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d21311161-Reviews-Mirakuru_Street_Food-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(4 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Internacional, Asiática</span></span><span class="biGQs _P VImYz ZNjnF">$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Fechado agora</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://mirakurubelem.wixsite.com/website" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Cardápio</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d21311161-Reviews-Mirakuru_Street_Food-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Comida excelente</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d21311161-Reviews-Mirakuru_Street_Food-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Experiência que vale à pena.</span></div></a></div></div></div></div></div></div></div><div class=""><div id="slot:728x90:inline1" class="Eufjb j u nFcpI" data-google-query-id="CNjRuvqL0pIDFfKKYQYdwkggjw"><div id="google_ads_iframe_/5349/ta.ta.br.s/sa.brazil.state_of_para_8__container__" style="border: 0pt none; display: inline-block; width: 728px; height: 90px;"><iframe frameborder="0" src="https://953e8c8f3ff1388a30f3a179ada483bf.safeframe.googlesyndication.com/safeframe/1-0-45/html/container.html" id="google_ads_iframe_/5349/ta.ta.br.s/sa.brazil.state_of_para_8" title="Conteúdo de anúncios de terceiros" name="" scrolling="no" marginwidth="0" marginheight="0" width="728" height="90" data-is-safeframe="true" sandbox="allow-forms allow-popups allow-popups-to-escape-sandbox allow-same-origin allow-scripts allow-top-navigation-by-user-activation" allow="private-state-token-redemption;attribution-reporting" aria-label="Publicidade" tabindex="0" data-google-container-id="3" style="border: 0px; vertical-align: bottom;" data-load-complete="true"></iframe></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d26491778-Reviews-Old_House_Blues_e_Jazz-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/f0/01/6f/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/f0/01/6f/caption.jpg?w=700&amp;h=700&amp;s=1 2x"><img width="400" height="400" alt="" loading="lazy" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/f0/01/6f/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/f0/01/6f/caption.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/f0/01/6f/caption.jpg?w=400&amp;h=400&amp;s=1"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/f0/01/76/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/f0/01/76/caption.jpg?w=700&amp;h=700&amp;s=1 2x"><img width="400" height="400" alt="." loading="lazy" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/f0/01/76/caption.jpg?w=400&amp;h=400&amp;s=1" style="" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/f0/01/76/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/f0/01/76/caption.jpg?w=800&amp;h=800&amp;s=1 2x"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/f0/01/75/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/f0/01/75/caption.jpg?w=700&amp;h=700&amp;s=1 2x"><img width="400" height="400" alt="." loading="lazy" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/f0/01/75/caption.jpg?w=400&amp;h=400&amp;s=1" style="" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/f0/01/75/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/f0/01/75/caption.jpg?w=800&amp;h=800&amp;s=1 2x"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-26491778"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d26491778-Reviews-Old_House_Blues_e_Jazz-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">4. Old House - Blues e Jazz</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u Q1"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4,2</span></div></span><div class="nKWJn u Q1 qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby="_lithium-r_bp_" data-automation="bubbleRatingImage"><title id="_lithium-r_bp_">4,2 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d26491778-Reviews-Old_House_Blues_e_Jazz-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(11 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Italiana, Bar</span></span><span class="biGQs _P VImYz ZNjnF">$$$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Fechado agora</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><button type="button" class="LuHAv Cj G_ B-"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Cardápio</span></span></button></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d26491778-Reviews-Old_House_Blues_e_Jazz-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Muito aconchegante</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d26491778-Reviews-Old_House_Blues_e_Jazz-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Experiência única em Belém</span></div></a></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d7252312-Reviews-Radisson_Hotel_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/26/5c/5f/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/26/5c/5f/caption.jpg?w=700&amp;h=700&amp;s=1 2x"><img width="400" height="400" alt="" loading="lazy" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/26/5c/5f/caption.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/26/5c/5f/caption.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/26/5c/5f/caption.jpg?w=400&amp;h=400&amp;s=1"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img width="100%" height="100%" alt="" loading="lazy" src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img width="100%" height="100%" alt="" loading="lazy" src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-7252312"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d7252312-Reviews-Radisson_Hotel_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">5. Radisson Hotel Belem</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u Q1"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>3,9</span></div></span><div class="nKWJn u Q1 qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby="_lithium-r_c2_" data-automation="bubbleRatingImage"><title id="_lithium-r_c2_">3,9 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d7252312-Reviews-Radisson_Hotel_Belem-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(9 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Internacional, Europeia</span></span><span class="biGQs _P VImYz ZNjnF"></span></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d7252312-Reviews-Radisson_Hotel_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Maravilhoso!</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d7252312-Reviews-Radisson_Hotel_Belem-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Magnífico </span></div></a></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d2373296-Reviews-Na_Telha-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/b0/e1/39/20180104-211513-largejpg.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/b0/e1/39/20180104-211513-largejpg.jpg?w=700&amp;h=700&amp;s=1 2x"><img width="400" height="400" alt="" loading="lazy" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/b0/e1/39/20180104-211513-largejpg.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/b0/e1/39/20180104-211513-largejpg.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/b0/e1/39/20180104-211513-largejpg.jpg?w=400&amp;h=400&amp;s=1"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img width="100%" height="100%" alt="" loading="lazy" src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img width="100%" height="100%" alt="" loading="lazy" src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-2373296"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d2373296-Reviews-Na_Telha-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">6. Na Telha</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u Q1"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4,2</span></div></span><div class="nKWJn u Q1 qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby="_lithium-r_cb_" data-automation="bubbleRatingImage"><title id="_lithium-r_cb_">4,2 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d2373296-Reviews-Na_Telha-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(142 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Brasileira, Frutos do mar</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Fecha em 16 minutos</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d2373296-Reviews-Na_Telha-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Almoço </span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d2373296-Reviews-Na_Telha-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Vale a experiência! </span></div></a></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d9558565-Reviews-Restaurante_Casa_Mia-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/22/4f/a9/df/fachada-da-nossa-casa.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/22/4f/a9/df/fachada-da-nossa-casa.jpg?w=700&amp;h=700&amp;s=1 2x"><img width="400" height="400" alt="" loading="lazy" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/22/4f/a9/df/fachada-da-nossa-casa.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/22/4f/a9/df/fachada-da-nossa-casa.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/22/4f/a9/df/fachada-da-nossa-casa.jpg?w=400&amp;h=400&amp;s=1"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img width="100%" height="100%" alt="" loading="lazy" src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img width="100%" height="100%" alt="" loading="lazy" src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-9558565"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d9558565-Reviews-Restaurante_Casa_Mia-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">7. Restaurante Casa Mia</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u Q1"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4,0</span></div></span><div class="nKWJn u Q1 qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby="_lithium-r_ck_" data-automation="bubbleRatingImage"><title id="_lithium-r_ck_">4 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d9558565-Reviews-Restaurante_Casa_Mia-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(167 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Brasileira, Café</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div><div class="Mi"><div class="ZvrsW N G biqBm"><span class="biGQs _P VImYz ZNjnF"><span class="f"><a target="_blank" href="https://cdn.flipsnack.com/widget/v2/widget.html?hash=t30v5s5hp8" class="BMQDV _F Gv wSSLS SwZTJ"><span class="SZrBo"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M16.719 8.193V19.97h-1.5v-4.124h-1.987V10.94c0-1.03.453-1.748 1.055-2.184a3 3 0 0 1 1.67-.552zm-1.5 6.153v-4.41l-.052.036c-.226.164-.435.435-.435.97v3.404z"></path><path d="M9.086 19.97h1.5v-7.72q.368-.094.672-.295c.382-.254.633-.6.796-.957.31-.682.337-1.488.337-2.043v-.75h-1.5v.75c0 .556-.04 1.065-.202 1.42a1 1 0 0 1-.103.178V8.205h-1.5v2.28a1 1 0 0 1-.068-.118c-.176-.344-.237-.85-.237-1.412v-.75h-1.5v.75c0 .604.054 1.414.4 2.093.18.353.446.686.833.927q.264.163.572.253z"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M18.817.317 4.136 4.492H4.02v19.191h15.96V4.492h-1.162zm-1.5 4.175v-2.19L9.62 4.493zm1.162 1.5H5.52v16.191h12.96z"></path></svg><span class="biGQs _P VImYz ZNjnF">Cardápio</span></span></a></span></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d9558565-Reviews-Restaurante_Casa_Mia-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">comida boa, atendimento demorado</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d9558565-Reviews-Restaurante_Casa_Mia-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Bom restaurante com área kids</span></div></a></div></div></div></div></div></div></div><div class=""><div id="slot:728x90:inline2" class="Eufjb j u nFcpI" data-google-query-id="CNnRuvqL0pIDFfKKYQYdwkggjw"><div id="google_ads_iframe_/5349/ta.ta.br.s/sa.brazil.state_of_para_9__container__" style="border: 0pt none; display: inline-block; width: 728px; height: 90px;"><iframe frameborder="0" src="https://953e8c8f3ff1388a30f3a179ada483bf.safeframe.googlesyndication.com/safeframe/1-0-45/html/container.html" id="google_ads_iframe_/5349/ta.ta.br.s/sa.brazil.state_of_para_9" title="Conteúdo de anúncios de terceiros" name="" scrolling="no" marginwidth="0" marginheight="0" width="728" height="90" data-is-safeframe="true" sandbox="allow-forms allow-popups allow-popups-to-escape-sandbox allow-same-origin allow-scripts allow-top-navigation-by-user-activation" allow="private-state-token-redemption;attribution-reporting" aria-label="Publicidade" tabindex="0" data-google-container-id="4" style="border: 0px; vertical-align: bottom;" data-load-complete="true"></iframe></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d2373295-Reviews-Estacao_Gourmet-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/06/19/89/34/estacao-gourmet.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/06/19/89/34/estacao-gourmet.jpg?w=700&amp;h=700&amp;s=1 2x"><img width="400" height="400" alt="" loading="lazy" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/06/19/89/34/estacao-gourmet.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/06/19/89/34/estacao-gourmet.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/06/19/89/34/estacao-gourmet.jpg?w=400&amp;h=400&amp;s=1"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img width="100%" height="100%" alt="" loading="lazy" src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img width="100%" height="100%" alt="" loading="lazy" src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-2373295"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d2373295-Reviews-Estacao_Gourmet-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">8. Estação Gourmet</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u Q1"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4,5</span></div></span><div class="nKWJn u Q1 qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby="_lithium-r_ct_" data-automation="bubbleRatingImage"><title id="_lithium-r_ct_">4,5 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d2373295-Reviews-Estacao_Gourmet-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(83 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Brasileira, Internacional</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Fechado agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d2373295-Reviews-Estacao_Gourmet-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Supimpa </span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d2373295-Reviews-Estacao_Gourmet-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Excelente opção de comida a quilo em Belém</span></div></a></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d10480155-Reviews-Vegas_Poker_Club-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/a2/b1/f9/20171229-183247-largejpg.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/a2/b1/f9/20171229-183247-largejpg.jpg?w=700&amp;h=700&amp;s=1 2x"><img width="400" height="400" alt="" loading="lazy" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/a2/b1/f9/20171229-183247-largejpg.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/a2/b1/f9/20171229-183247-largejpg.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/a2/b1/f9/20171229-183247-largejpg.jpg?w=400&amp;h=400&amp;s=1"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img width="100%" height="100%" alt="" loading="lazy" src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img width="100%" height="100%" alt="" loading="lazy" src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-10480155"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d10480155-Reviews-Vegas_Poker_Club-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">9. Vegas Poker Club</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u Q1"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>4,1</span></div></span><div class="nKWJn u Q1 qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby="_lithium-r_d6_" data-automation="bubbleRatingImage"><title id="_lithium-r_d6_">4,1 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d10480155-Reviews-Vegas_Poker_Club-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(38 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Brasileira, Bar</span></span><span class="biGQs _P VImYz ZNjnF">$$ - $$$</span></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d10480155-Reviews-Vegas_Poker_Club-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Escutar uma boa música tomando um chopp!</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d10480155-Reviews-Vegas_Poker_Club-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Legal para tomar um chopp</span></div></a></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d12316068-Reviews-Le_Bistrot-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/13/02/d5/e9/nossa-logomarca-desde.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/13/02/d5/e9/nossa-logomarca-desde.jpg?w=700&amp;h=-1&amp;s=1 2x"><img width="400" height="400" alt="" loading="lazy" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/13/02/d5/e9/nossa-logomarca-desde.jpg?w=400&amp;h=-1&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/13/02/d5/e9/nossa-logomarca-desde.jpg?w=800&amp;h=-1&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/13/02/d5/e9/nossa-logomarca-desde.jpg?w=400&amp;h=-1&amp;s=1"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img width="100%" height="100%" alt="" loading="lazy" src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img width="100%" height="100%" alt="" loading="lazy" src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-12316068"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d12316068-Reviews-Le_Bistrot-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">10. Le Bistrot</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u Q1"><span aria-hidden="true"><div class="biGQs _P VImYz ZNjnF" data-automation="bubbleRatingValue"><span>3,7</span></div></span><div class="nKWJn u Q1 qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby="_lithium-r_df_" data-automation="bubbleRatingImage"><title id="_lithium-r_df_">3,7 de 5 círculos</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12z" transform="translate(52 0)"></path><path d="M 12 0C5.389 0 0 5.389 0 12c0 6.62 5.389 12 12 12 6.62 0 12-5.379 12-12S18.621 0 12 0zm0 2a9.984 9.984 0 0110 10 9.976 9.976 0 01-10 10z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d12316068-Reviews-Le_Bistrot-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(26 avaliações) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Francesa, Internacional</span></span><span class="biGQs _P VImYz ZNjnF"></span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Fechado agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div><div class="MUVLp f e WVNdl"><a target="_blank" href="/Restaurant_Review-g303404-d12316068-Reviews-Le_Bistrot-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Um lugar acolhedor</span></div></a><a target="_blank" href="/Restaurant_Review-g303404-d12316068-Reviews-Le_Bistrot-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P VImYz alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU ZNjnF"><span class="oTOkl">Boa opção em Belém </span></div></a></div></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d13435710-Reviews-Bar_e_Restaurante_La_Marine-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><div class="IdURT w carousel UznXc wSSLS" data-automation="photoCarousel"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R mdkdE afQPz eXZKw"><source media="(max-width: 1023px)" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/b4/56/dd/voce-pode-almocar-em.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/b4/56/dd/voce-pode-almocar-em.jpg?w=700&amp;h=700&amp;s=1 2x"><img width="400" height="400" alt="" loading="lazy" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/b4/56/dd/voce-pode-almocar-em.jpg?w=400&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/b4/56/dd/voce-pode-almocar-em.jpg?w=800&amp;h=800&amp;s=1 2x" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/b4/56/dd/voce-pode-almocar-em.jpg?w=400&amp;h=400&amp;s=1"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img width="100%" height="100%" alt="" loading="lazy" src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" style="object-fit: fill;"></picture></li><li class="CyFNY _A"><picture class="NhWcC _R mdkdE afQPz eXZKw"><img width="100%" height="100%" alt="" loading="lazy" src="data:image/svg+xml;utf8,%3C%3Fxml%20version%3D%221.0%22%3F%3E%3Csvg%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20style%3D%22background%3A%23DFE1DF%22%3E%3Csvg%20x%3D%2250%25%22%20y%3D%2250%25%22%20width%3D%2250%22%20height%3D%2250%22%20viewBox%3D%220%200%2040%2040%22%20xmlns%3D%22http%3A%2F%2Fwww.w3.org%2F2000%2Fsvg%22%20overflow%3D%22visible%22%3E%3Cg%20transform%3D%22translate(-20%2C%20-20)%22%3E%3Cpath%20d%3D%22M28.8%204.53333C32.4819%204.53333%2035.4667%207.5181%2035.4667%2011.2C35.4667%2014.8819%2032.4819%2017.8667%2028.8%2017.8667C25.1181%2017.8667%2022.1333%2014.8819%2022.1333%2011.2C22.1333%207.5181%2025.1181%204.53333%2028.8%204.53333Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3Cpath%20fill-rule%3D%22evenodd%22%20clip-rule%3D%22evenodd%22%20d%3D%22M37.5717%200.00078125C38.9186%200.034925%2040%201.13748%2040%202.49258V37.5074L39.9992%2037.5717C39.9656%2038.8972%2038.8972%2039.9656%2037.5717%2039.9992L37.5074%2040H2.49258L2.42826%2039.9992C1.10276%2039.9656%200.0343837%2038.8972%200.00078125%2037.5717L0%2037.5074V2.49258C2.78546e-05%201.13748%201.08138%200.0349255%202.42826%200.00078125L2.49258%200H37.5074L37.5717%200.00078125ZM2.49258%202.4C2.44146%202.40003%202.40003%202.44146%202.4%202.49258V23.4285C4.22877%2021.3137%206.98148%2019.4667%2010.6667%2019.4667C17.9151%2019.4667%2019.4369%2029.2834%2037.6%2030.0815V2.49258C37.6%202.44146%2037.5585%202.40003%2037.5074%202.4H2.49258Z%22%20fill%3D%22%23F7F7F7%22%2F%3E%3C%2Fg%3E%3C%2Fsvg%3E%3C%2Fsvg%3E" style="object-fit: fill;"></picture></li></ul></div><div class="AWPqO Z1 _S sagSj cSbfm"><button class="BrOJk u j z _F _S wSSLS HuPlH gSpCh IyzRb" type="button" aria-label="Foto anterior" disabled="" data-automation="leftNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m10.736 20.792-8.58-8.58 8.58-8.582 1.06 1.061-6.77 6.77h16.08v1.5H5.026l6.77 6.77z"></path></svg></button></div><div class="AWPqO Z1 _S nlmjL"><button class="BrOJk u j z _F _S wSSLS HuPlH IyzRb" type="button" aria-label="Próxima foto" data-automation="rightNavArrow"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="m12.631 3.632 8.578 8.577-8.58 8.58-1.062-1.06 6.77-6.77-16.075.004v-1.5l16.076-.004-6.767-6.767z"></path></svg></button></div><div class="LbOhq"><div class="xrpyC l _U f u j rFcwj"><div class="CKBaw" data-clicksource="IndicatorDot" data-automation="indicatorDots-0" style="width: 126px;"><div class="gqkHJ u f HfKGv" style="transform: translate3d(0px, 0px, 0px) scale3d(0, 0, 1);"></div><div class="gqkHJ u f sWDGS HfKGv" style="transform: translate3d(20px, 0px, 0px) scale3d(1.3, 1.3, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(40px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(60px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(80px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(100px, 0px, 0px) scale3d(1, 1, 1);"></div><div class="gqkHJ u f HfKGv" style="transform: translate3d(120px, 0px, 0px) scale3d(0, 0, 1);"></div></div></div></div></div></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-13435710"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d13435710-Reviews-Bar_e_Restaurante_La_Marine-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">11. Bar e Restaurante La Marine</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u Q1"><div class="nKWJn u Q1 qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby="_lithium-r_do_" data-automation="bubbleRatingImage"><title id="_lithium-r_do_">Ainda não há avaliações</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d13435710-Reviews-Bar_e_Restaurante_La_Marine-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(0 avaliação) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Francesa, Brasileira</span></span><span class="biGQs _P VImYz ZNjnF">$</span></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div class="bGLbH"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ QHaGY" href="/UserReviewEdit-g303404-d13435710-Reviews-Bar_e_Restaurante_La_Marine-Belem_State_of_Para.html"><span class="biGQs _P ezezH"><div class="tWvGj"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M13.788 7.293 6.19 14.868l-.816 3.746 3.724-.839 7.588-7.583zm3.96 1.84-2.898-2.9.556-.554A2.32 2.32 0 0 1 17.02 5c.378 0 .73.104 1.031.315l.01.007.012.008c1.12.757 1.221 2.26.326 3.151zm-7.896 10.01-5.99 1.35q-.032.009-.064.007a.297.297 0 0 1-.29-.36l1.31-6.023 9.529-9.5A3.82 3.82 0 0 1 17.02 3.5c.66 0 1.318.184 1.893.587a3.536 3.536 0 0 1 .546 5.457z"></path></svg>Faça uma avaliação </div></span></a></div></div></div></div></div></div><div class=""><div id="slot:728x90:inline3" class="Eufjb j u nFcpI" data-google-query-id="CNrRuvqL0pIDFfKKYQYdwkggjw"><div id="google_ads_iframe_/5349/ta.ta.br.s/sa.brazil.state_of_para_10__container__" style="border: 0pt none; display: inline-block; width: 728px; height: 90px;"><iframe frameborder="0" src="https://953e8c8f3ff1388a30f3a179ada483bf.safeframe.googlesyndication.com/safeframe/1-0-45/html/container.html" id="google_ads_iframe_/5349/ta.ta.br.s/sa.brazil.state_of_para_10" title="Conteúdo de anúncios de terceiros" name="" scrolling="no" marginwidth="0" marginheight="0" width="728" height="90" data-is-safeframe="true" sandbox="allow-forms allow-popups allow-popups-to-escape-sandbox allow-same-origin allow-scripts allow-top-navigation-by-user-activation" allow="private-state-token-redemption;attribution-reporting" aria-label="Publicidade" tabindex="0" data-google-container-id="5" style="border: 0px; vertical-align: bottom;" data-load-complete="true"></iframe></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d27167379-Reviews-Dalie_Mediterrane-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><svg xmlns="http://www.w3.org/2000/svg" class="QgHgJ"><svg x="50%" y="50%" width="50" height="50" viewBox="0 0 40 40" xmlns="http://www.w3.org/2000/svg" overflow="visible"><g transform="translate(-20, -20)"><path d="M28.8 4.53333C32.4819 4.53333 35.4667 7.5181 35.4667 11.2C35.4667 14.8819 32.4819 17.8667 28.8 17.8667C25.1181 17.8667 22.1333 14.8819 22.1333 11.2C22.1333 7.5181 25.1181 4.53333 28.8 4.53333Z" fill="var(--scheme-inverse-on-surface)"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M37.5717 0.00078125C38.9186 0.034925 40 1.13748 40 2.49258V37.5074L39.9992 37.5717C39.9656 38.8972 38.8972 39.9656 37.5717 39.9992L37.5074 40H2.49258L2.42826 39.9992C1.10276 39.9656 0.0343837 38.8972 0.00078125 37.5717L0 37.5074V2.49258C2.78546e-05 1.13748 1.08138 0.0349255 2.42826 0.00078125L2.49258 0H37.5074L37.5717 0.00078125ZM2.49258 2.4C2.44146 2.40003 2.40003 2.44146 2.4 2.49258V23.4285C4.22877 21.3137 6.98148 19.4667 10.6667 19.4667C17.9151 19.4667 19.4369 29.2834 37.6 30.0815V2.49258C37.6 2.44146 37.5585 2.40003 37.5074 2.4H2.49258Z" fill="var(--scheme-inverse-on-surface)"></path></g></svg></svg></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-27167379"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d27167379-Reviews-Dalie_Mediterrane-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">12. Dalie Mediterrané</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u Q1"><div class="nKWJn u Q1 qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby="_lithium-r_e1_" data-automation="bubbleRatingImage"><title id="_lithium-r_e1_">Ainda não há avaliações</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d27167379-Reviews-Dalie_Mediterrane-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(0 avaliação) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Internacional, Mediterrânea</span></span><span class="biGQs _P VImYz ZNjnF"></span></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div class="bGLbH"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ QHaGY" href="/UserReviewEdit-g303404-d27167379-Reviews-Dalie_Mediterrane-Belem_State_of_Para.html"><span class="biGQs _P ezezH"><div class="tWvGj"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M13.788 7.293 6.19 14.868l-.816 3.746 3.724-.839 7.588-7.583zm3.96 1.84-2.898-2.9.556-.554A2.32 2.32 0 0 1 17.02 5c.378 0 .73.104 1.031.315l.01.007.012.008c1.12.757 1.221 2.26.326 3.151zm-7.896 10.01-5.99 1.35q-.032.009-.064.007a.297.297 0 0 1-.29-.36l1.31-6.023 9.529-9.5A3.82 3.82 0 0 1 17.02 3.5c.66 0 1.318.184 1.893.587a3.536 3.536 0 0 1 .546 5.457z"></path></svg>Faça uma avaliação </div></span></a></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d33354040-Reviews-The_Coffee_Batista_Campos-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><svg xmlns="http://www.w3.org/2000/svg" class="QgHgJ"><svg x="50%" y="50%" width="50" height="50" viewBox="0 0 40 40" xmlns="http://www.w3.org/2000/svg" overflow="visible"><g transform="translate(-20, -20)"><path d="M28.8 4.53333C32.4819 4.53333 35.4667 7.5181 35.4667 11.2C35.4667 14.8819 32.4819 17.8667 28.8 17.8667C25.1181 17.8667 22.1333 14.8819 22.1333 11.2C22.1333 7.5181 25.1181 4.53333 28.8 4.53333Z" fill="var(--scheme-inverse-on-surface)"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M37.5717 0.00078125C38.9186 0.034925 40 1.13748 40 2.49258V37.5074L39.9992 37.5717C39.9656 38.8972 38.8972 39.9656 37.5717 39.9992L37.5074 40H2.49258L2.42826 39.9992C1.10276 39.9656 0.0343837 38.8972 0.00078125 37.5717L0 37.5074V2.49258C2.78546e-05 1.13748 1.08138 0.0349255 2.42826 0.00078125L2.49258 0H37.5074L37.5717 0.00078125ZM2.49258 2.4C2.44146 2.40003 2.40003 2.44146 2.4 2.49258V23.4285C4.22877 21.3137 6.98148 19.4667 10.6667 19.4667C17.9151 19.4667 19.4369 29.2834 37.6 30.0815V2.49258C37.6 2.44146 37.5585 2.40003 37.5074 2.4H2.49258Z" fill="var(--scheme-inverse-on-surface)"></path></g></svg></svg></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-33354040"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d33354040-Reviews-The_Coffee_Batista_Campos-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">13. The Coffee Batista Campos</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u Q1"><div class="nKWJn u Q1 qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby="_lithium-r_ea_" data-automation="bubbleRatingImage"><title id="_lithium-r_ea_">Ainda não há avaliações</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d33354040-Reviews-The_Coffee_Batista_Campos-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(0 avaliação) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Café e chá, Japonesa</span></span><span class="biGQs _P VImYz ZNjnF"></span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f lFNDg"><span class="biGQs _P ZNjnF">Aberto agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div class="bGLbH"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ QHaGY" href="/UserReviewEdit-g303404-d33354040-Reviews-The_Coffee_Batista_Campos-Belem_State_of_Para.html"><span class="biGQs _P ezezH"><div class="tWvGj"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M13.788 7.293 6.19 14.868l-.816 3.746 3.724-.839 7.588-7.583zm3.96 1.84-2.898-2.9.556-.554A2.32 2.32 0 0 1 17.02 5c.378 0 .73.104 1.031.315l.01.007.012.008c1.12.757 1.221 2.26.326 3.151zm-7.896 10.01-5.99 1.35q-.032.009-.064.007a.297.297 0 0 1-.29-.36l1.31-6.023 9.529-9.5A3.82 3.82 0 0 1 17.02 3.5c.66 0 1.318.184 1.893.587a3.536 3.536 0 0 1 .546 5.457z"></path></svg>Faça uma avaliação </div></span></a></div></div></div></div></div></div><div data-automation="restaurantCard"><div class="tbrcR _T DxHsn TwZIp rrkMt nSZNd DALUy Re"><div class="vkMWZ _T M0 Fl y M0"><div class="EjTgf w"><div></div><a href="/Restaurant_Review-g303404-d24941804-Reviews-Delicatesse-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt PaRlG"><div class="WcLPg _Z"><div class="afQPz SyjMH ttWhi o NlJeq w _Z Gm A"><div class="OCjqp w _Z"><div class="_T w _Z" data-clicksource="Photo"><svg xmlns="http://www.w3.org/2000/svg" class="QgHgJ"><svg x="50%" y="50%" width="50" height="50" viewBox="0 0 40 40" xmlns="http://www.w3.org/2000/svg" overflow="visible"><g transform="translate(-20, -20)"><path d="M28.8 4.53333C32.4819 4.53333 35.4667 7.5181 35.4667 11.2C35.4667 14.8819 32.4819 17.8667 28.8 17.8667C25.1181 17.8667 22.1333 14.8819 22.1333 11.2C22.1333 7.5181 25.1181 4.53333 28.8 4.53333Z" fill="var(--scheme-inverse-on-surface)"></path><path fill-rule="evenodd" clip-rule="evenodd" d="M37.5717 0.00078125C38.9186 0.034925 40 1.13748 40 2.49258V37.5074L39.9992 37.5717C39.9656 38.8972 38.8972 39.9656 37.5717 39.9992L37.5074 40H2.49258L2.42826 39.9992C1.10276 39.9656 0.0343837 38.8972 0.00078125 37.5717L0 37.5074V2.49258C2.78546e-05 1.13748 1.08138 0.0349255 2.42826 0.00078125L2.49258 0H37.5074L37.5717 0.00078125ZM2.49258 2.4C2.44146 2.40003 2.40003 2.44146 2.4 2.49258V23.4285C4.22877 21.3137 6.98148 19.4667 10.6667 19.4667C17.9151 19.4667 19.4369 29.2834 37.6 30.0815V2.49258C37.6 2.44146 37.5585 2.40003 37.5074 2.4H2.49258Z" fill="var(--scheme-inverse-on-surface)"></path></g></svg></svg></div></div><div class="cnQjJ z REnnF"><div class="qijCB _Q"><button type="button" aria-label="Salvar em uma viagem" class="vKDXe u j _T z _S _F xwbQG cunPs jODdB" data-automation="trips-save-button-item-24941804"><svg viewBox="0 0 24 24" width="24px" height="24px" class="d Vb icjEL" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M3.798 5.166A5.77 5.77 0 0 1 7.72 3.63c1.455 0 2.857.548 3.922 1.536l.005.005.341.322.332-.317a5.77 5.77 0 0 1 3.928-1.54c1.458 0 2.862.55 3.928 1.54l.004.004c1.093 1.032 1.598 2.324 1.569 3.662-.03 1.323-.579 2.643-1.5 3.785-.884 1.096-2.85 2.943-4.547 4.478a185 185 0 0 1-3.153 2.785l-.069.059-.489-.569.489.569-.485.416-.488-.412a102 102 0 0 1-7.75-7.288l-.021-.021-.02-.023c-1.725-2.115-2.203-5.32.08-7.453zm8.19 13.226.472-.412a184 184 0 0 0 2.236-1.988c1.72-1.556 3.59-3.32 4.385-4.306.757-.939 1.147-1.948 1.168-2.877.02-.912-.313-1.795-1.097-2.536a4.27 4.27 0 0 0-2.904-1.138c-1.08 0-2.117.407-2.903 1.136l-1.35 1.292-1.375-1.3a4.27 4.27 0 0 0-2.9-1.133 4.27 4.27 0 0 0-2.901 1.135c-1.507 1.408-1.353 3.659.042 5.385a101 101 0 0 0 7.127 6.742"></path></svg></button></div></div></div></div></a></div></div><div class="XIWnB z y M0"><div class="IcVzi y M0 _T"><div class="vwOkl"><div><div class="mfKvs f e"><div class="UIwAG f k"><div class="KBZbF f e"><div class="ZvrsW N G"><a href="/Restaurant_Review-g303404-d24941804-Reviews-Delicatesse-Belem_State_of_Para.html" class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"><div class="biGQs _P SewaP alXOW oCpZu GzNcM nvOhm UTQMg ZTpaU mtnKn OgHoE">14. Délicatesse</div></a><div class="kzrsh"><div class="VVbkp"><div class="MyMKp u Q1"><div class="nKWJn u Q1 qMyjI"><svg class="evwcZ" viewBox="0 0 128 24" width="68" height="12" aria-labelledby="_lithium-r_ej_" data-automation="bubbleRatingImage"><title id="_lithium-r_ej_">Ainda não há avaliações</title><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform=""></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(26 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(52 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(78 0)"></path><path d="M 12 0C5.388 0 0 5.388 0 12s5.388 12 12 12 12-5.38 12-12c0-6.612-5.38-12-12-12zm0 2a9.983 9.983 0 019.995 10 10 10 0 01-10 10A10 10 0 012 12 10 10 0 0112 2z" transform="translate(104 0)"></path></svg><div class="biGQs _P VImYz ZNjnF"><a target="_self" href="/Restaurant_Review-g303404-d24941804-Reviews-Delicatesse-Belem_State_of_Para.html#REVIEWS" class="BMQDV _F Gv wSSLS SwZTJ FGwzt suezE"><div class="biGQs _P ZNjnF" data-automation="bubbleReviewCount"><span>(0 avaliação) </span></div></a></div></div></div></div></div></div></div></div><div><div class="ZvrsW N G"><div class="ZvrsW N G biqBm"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M14.051 6.549v.003l1.134 1.14 3.241-3.25.003-.002 1.134 1.136-3.243 3.252 1.134 1.14a1 1 0 0 0 .09-.008c.293-.05.573-.324.72-.474l.005-.006 2.596-2.603L22 8.016l-2.597 2.604a3.73 3.73 0 0 1-1.982 1.015 4.3 4.3 0 0 1-3.162-.657l-.023-.016-.026-.018-1.366 1.407 8.509 8.512L20.219 22l-.002-.002-6.654-6.663-2.597 2.76-7.3-7.315C1.967 8.948 1.531 6.274 2.524 4.198c.241-.504.566-.973.978-1.386l8.154 8.416 1.418-1.423-.039-.045c-.858-1.002-1.048-2.368-.62-3.595a4.15 4.15 0 0 1 .983-1.561L16 2l1.135 1.138-2.598 2.602-.047.045c-.16.151-.394.374-.433.678zM3.809 5.523c-.362 1.319-.037 2.905 1.06 4.103L10.93 15.7l1.408-1.496zM2.205 20.697 3.34 21.84l4.543-4.552-1.135-1.143z"></path></svg><span class="biGQs _P VImYz ZNjnF">Steakhouse, Brasileira</span></span><span class="biGQs _P VImYz ZNjnF"></span><div class="cJyep f k K"><span class="f"><svg viewBox="0 0 24 24" width="16px" height="16px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3.75a8.25 8.25 0 1 0 0 16.5 8.25 8.25 0 0 0 0-16.5M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12m9-5.5h1.5v5.19l3.28 3.28-1.06 1.06-3.72-3.72z"></path></svg><div class="Pwlnc f RAyMq"><span class="biGQs _P ZNjnF">Fechado agora</span></div></span></div></div></div></div></div></div></div></div><div class="CnVYs y M0 _T"><div class="aXNmm e _Z"><div class="bGLbH"><a class="rmyCe _G B- z _S c Wc wSSLS jWkoZ QHaGY" href="/UserReviewEdit-g303404-d24941804-Reviews-Delicatesse-Belem_State_of_Para.html"><span class="biGQs _P ezezH"><div class="tWvGj"><svg viewBox="0 0 24 24" width="20px" height="20px" class="d Vb egaXP UmNoP" aria-hidden="true"><path fill-rule="evenodd" clip-rule="evenodd" d="M13.788 7.293 6.19 14.868l-.816 3.746 3.724-.839 7.588-7.583zm3.96 1.84-2.898-2.9.556-.554A2.32 2.32 0 0 1 17.02 5c.378 0 .73.104 1.031.315l.01.007.012.008c1.12.757 1.221 2.26.326 3.151zm-7.896 10.01-5.99 1.35q-.032.009-.064.007a.297.297 0 0 1-.29-.36l1.31-6.023 9.529-9.5A3.82 3.82 0 0 1 17.02 3.5c.66 0 1.318.184 1.893.587a3.536 3.536 0 0 1 .546 5.457z"></path></svg>Faça uma avaliação </div></span></a></div></div></div></div></div></div><div class="qAZoU c"><div class="GMYGA"><div class="mkNRT j"></div><div class="LaMdn j"><div class="aMnVW"><div class="Yzhnw P"><button class="BrOJk u j z _F _S wSSLS tIqAi iNBVo SSqtP" type="button" disabled="" aria-label="1"><span class="kLqdM"><span class="biGQs _P ezezH">1</span></span></button></div></div></div></div></div><div class="qAZoU c"><div class="biGQs _P VImYz ZNjnF"><div class="Ci">Mostrando 1–14 de 14 resultados</div></div></div></div>
"""
soup_completo = BeautifulSoup(html_completo_exemplos, 'html.parser')

# Encontrar todos os blocos de restaurante
# A classe 'tbrcR' combinada com outras parece ser um bom seletor para cada "cartão" de restaurante.
restaurant_blocks = soup_completo.find_all('div', class_='tbrcR')

print("Iniciando a extração detalhada de cada restaurante:")
restaurantes_detalhados = [] # Lista para armazenar os dados estruturados

for block in restaurant_blocks:

    nome_link_tag = block.find('a', class_=lambda c: c and 'BMQDV' in c and 'ukgos' in c)
    nome = "Nome não encontrado"
    if nome_link_tag:
        nome_div_tag = nome_link_tag.find('div', class_=lambda c: c and 'biGQs' in c and 'fiohW' in c)
        if nome_div_tag:
            nome = nome_div_tag.get_text(strip=True)

    reviews_tag = block.find('div', {'data-automation': 'bubbleReviewCount'})
    reviews = reviews_tag.find('span').get_text(strip=True) if reviews_tag and reviews_tag.find('span') else "Reviews não encontrado"

    rating_tag = block.find('div', {'data-automation': 'bubbleRatingValue'})
    rating = rating_tag.find('span').get_text(strip=True) if rating_tag and rating_tag.find('span') else "Avaliação não encontrada"

    culinaria_preco_div = block.find('div', class_=lambda c: c and 'ZvrsW' in c and 'biqBm' in c)
    
    tipo_culinaria = "Tipo de Culinária não encontrado"
    preco = "Preço não encontrado"

    if culinaria_preco_div:

        spans_info = culinaria_preco_div.find_all('span', class_=lambda c: c and 'biGQs' in c and 'pZUbB' in c)
        
        if len(spans_info) > 1:
            tipo_culinaria = spans_info[0].get_text(strip=True)

        for i in range(1, len(spans_info)):
            text = spans_info[i].get_text(strip=True)
            if '$' in text:
                preco = text
                break

    localizacao = "Localização não especificada (do nome)"
    if ' - ' in nome:
        partes_nome = nome.split(' - ')
        if len(partes_nome) > 1:
            localizacao = partes_nome[-1].strip()

    link = "Link não encontrado"
    if nome_link_tag and 'href' in nome_link_tag.attrs:
        link = "https://www.tripadvisor.com.br" + nome_link_tag['href']

    restaurantes_detalhados.append({
        "Nome": nome,
        "Avaliacao": rating,
        "Reviews": reviews,
        "Preco": preco,
        "Tipo Culinária": tipo_culinaria,
        "Localizacao": localizacao,
        "Link": link
    })


if restaurantes_detalhados:
    print(f"\n--- {len(restaurantes_detalhados)} Restaurantes Extraidos do Tripadvisor ---")
    for i, r in enumerate(restaurantes_detalhados):
        print(f"--- Restaurante #{i+1} ---")
        for key, value in r.items():
            print(f"{key}: {r[key]}")
        print("-" * 40)
else:
    print("Nenhum detalhe de restaurante foi extraido. **Verifique os seletores HTML no Bloco 4**.")

Iniciando a extração detalhada de cada restaurante:

--- 14 Restaurantes Extraidos do Tripadvisor ---
--- Restaurante #1 ---
Nome: Nome não encontrado
Avaliacao: 4,9
Reviews: (1.699 avaliações)
Preco: Preço não encontrado
Tipo Culinária: Tipo de Culinária não encontrado
Localizacao: Localização não especificada (do nome)
Link: Link não encontrado
----------------------------------------
--- Restaurante #2 ---
Nome: Nome não encontrado
Avaliacao: 4,8
Reviews: (12 avaliações)
Preco: Preço não encontrado
Tipo Culinária: Tipo de Culinária não encontrado
Localizacao: Localização não especificada (do nome)
Link: Link não encontrado
----------------------------------------
--- Restaurante #3 ---
Nome: Nome não encontrado
Avaliacao: 5,0
Reviews: (4 avaliações)
Preco: Preço não encontrado
Tipo Culinária: Tipo de Culinária não encontrado
Localizacao: Localização não especificada (do nome)
Link: Link não encontrado
----------------------------------------
--- Restaurante #4 ---
Nome: Nome não enc